In [3]:
!git clone https://github.com/openai/shap-e

Cloning into 'shap-e'...
remote: Enumerating objects: 304, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 304 (delta 19), reused 23 (delta 11), pack-reused 256
Receiving objects: 100% (304/304), 11.71 MiB | 21.64 MiB/s, done.
Resolving deltas: 100% (19/19), done.


In [7]:
cd /content/shap-e

/content/shap-e


In [ ]:
!pip install -e .

In [25]:
import torch

from shap_e.diffusion.sample import sample_latents
from shap_e.diffusion.gaussian_diffusion import diffusion_from_config
from shap_e.models.download import load_model, load_config
from shap_e.util.notebooks import create_pan_cameras, decode_latent_images, gif_widget
from shap_e.util.image_util import load_image
from shap_e.util.notebooks import decode_latent_mesh

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xm = load_model('transmitter', device=device)
model = load_model('image300M', device=device)
diffusion = diffusion_from_config(load_config('diffusion'))

In [15]:
!wget https://www.seidenblumen-ross.de/media/image/c9/fc/03/Edelweiss_28cm_DP_Kunstblumen_kuenstliche_Blumen.jpg

--2023-05-07 21:01:19--  https://www.seidenblumen-ross.de/media/image/c9/fc/03/Edelweiss_28cm_DP_Kunstblumen_kuenstliche_Blumen.jpg
Resolving www.seidenblumen-ross.de (www.seidenblumen-ross.de)... 78.46.140.71
Connecting to www.seidenblumen-ross.de (www.seidenblumen-ross.de)|78.46.140.71|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 74442 (73K) [image/jpeg]
Saving to: ‘Edelweiss_28cm_DP_Kunstblumen_kuenstliche_Blumen.jpg’

Edelweiss_28cm_DP_K 100%[===================>]  72.70K   320KB/s    in 0.2s    

2023-05-07 21:01:20 (320 KB/s) - ‘Edelweiss_28cm_DP_Kunstblumen_kuenstliche_Blumen.jpg’ saved [74442/74442]



In [17]:
batch_size = 4
guidance_scale = 3.0

image = load_image("/content/shap-e/Edelweiss_28cm_DP_Kunstblumen_kuenstliche_Blumen.jpg")

latents = sample_latents(
    batch_size=batch_size,
    model=model,
    diffusion=diffusion,
    guidance_scale=guidance_scale,
    model_kwargs=dict(images=[image] * batch_size),
    progress=True,
    clip_denoised=True,
    use_fp16=True,
    use_karras=True,
    karras_steps=64,
    sigma_min=1e-3,
    sigma_max=160,
    s_churn=0,
)

  0%|          | 0/64 [00:00<?, ?it/s]

In [26]:
for i, latent in enumerate(latents):
    with open(f'example_mesh_{i}.ply', 'wb') as f:
        decode_latent_mesh(xm, latent).tri_mesh().write_ply(f)

/content/shap-e/shap_e/models/stf/renderer.py:286: UserWarning: exception rendering with PyTorch3D: No module named 'pytorch3d'
  warnings.warn(f"exception rendering with PyTorch3D: {exc}")
/content/shap-e/shap_e/models/stf/renderer.py:287: UserWarning: falling back on native PyTorch renderer, which does not support full gradients
  warnings.warn(


In [ ]:
render_mode = 'nerf' # you can change this to 'stf' for mesh rendering
size = 256 # this is the size of the renders; higher values take longer to render.

cameras = create_pan_cameras(size, device)
for i, latent in enumerate(latents):
    images = decode_latent_images(xm, latent, cameras, rendering_mode=render_mode)
    display(gif_widget(images))